In [ ]:
#pip install xgboost

In [1]:
import os

In [2]:
os.chdir('C:/Users/rohit/ISB/Capstone')
os.getcwd()

'C:\\Users\\rohit\\ISB\\Capstone'

In [3]:
%matplotlib inline

import numpy as np
import pandas as pd
import xgboost as xgb
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
dataset = pd.read_csv('summarized_data.csv')

In [5]:
dataset.head()

,Col_PK,Item_No,Location_Code,Purchase_MRP,Prevailing_MRP_as_on_Stock_Date,State,Region,Stock_Quantity,Days_In_Stock,Week_Start,...,Case_Material,Watchband_Strap_Type,Dial_Color,Case_Shape,Case_Size,Sales_Quantiy_In_Week,Days_To_Sell (sale date-receipt date),Avg_Billing_In_Week (avg of billing as per column O),"Purchase_Quantity (from purchase file, quantity by joining location and item)",Total_Available_Qty (H + AB)
0,5106817_S02,5106817,S02,63140.0,66300.0,ST05,West,1,2159 days 00:00:00.000000000,24-11-2019,...,STEEL,STEEL,WHITE,RECTANGULAR,32 mm,0,0 days 00:00:00.000000000,0.0,0,1
1,5106817_S23,5106817,S23,62500.0,66300.0,ST12,West,1,2351 days 00:00:00.000000000,24-11-2019,...,STEEL,STEEL,WHITE,RECTANGULAR,32 mm,0,0 days 00:00:00.000000000,0.0,0,1
2,5106817_S24,5106817,S24,48100.0,66300.0,ST16,South,1,2121 days 00:00:00.000000000,24-11-2019,...,STEEL,STEEL,WHITE,RECTANGULAR,32 mm,0,0 days 00:00:00.000000000,0.0,0,1
3,5106817_S62,5106817,S62,56500.0,66300.0,ST02,North,1,2298 days 00:00:00.000000000,24-11-2019,...,STEEL,STEEL,WHITE,RECTANGULAR,32 mm,0,0 days 00:00:00.000000000,0.0,0,1
4,5107206_S02,5107206,S02,142500.0,154100.0,ST05,West,1,254 days 00:00:00.000000000,24-11-2019,...,STEEL,LEATHER,SILVER,ROUND,40 mm,0,0 days 00:00:00.000000000,0.0,0,1


In [6]:
dataset.shape

(140749, 29)

In [7]:
dataset = pd.DataFrame(dataset)

In [8]:
dataset1= dataset.drop(columns=['Col_PK', 'Location_Code','State','Store_Code'])

In [9]:
dataset1.shape

(140749, 25)

In [10]:
dataset2=pd.get_dummies(dataset1, columns=['Region','Store_type','Store_location','City_type','Brand','Movement','Case_Material','Watchband_Strap_Type','Dial_Color','Case_Shape','Case_Size'])
dataset2.head()

,Item_No,Purchase_MRP,Prevailing_MRP_as_on_Stock_Date,Stock_Quantity,Days_In_Stock,Week_Start,Month,Year,No_,Sales_Quantiy_In_Week,...,Case_Size_57 mm,Case_Size_60 mm,Case_Size_<20 MM,Case_Size_<20 mm,Case_Size_>45 MM,Case_Size_>45 mm,Case_Size_M,Case_Size_S,Case_Size_da,Case_Size_ 44 MM
0,5106817,63140.0,66300.0,1,2159 days 00:00:00.000000000,24-11-2019,11,2019,5106817,0,...,0,0,0,0,0,0,0,0,0,0
1,5106817,62500.0,66300.0,1,2351 days 00:00:00.000000000,24-11-2019,11,2019,5106817,0,...,0,0,0,0,0,0,0,0,0,0
2,5106817,48100.0,66300.0,1,2121 days 00:00:00.000000000,24-11-2019,11,2019,5106817,0,...,0,0,0,0,0,0,0,0,0,0
3,5106817,56500.0,66300.0,1,2298 days 00:00:00.000000000,24-11-2019,11,2019,5106817,0,...,0,0,0,0,0,0,0,0,0,0
4,5107206,142500.0,154100.0,1,254 days 00:00:00.000000000,24-11-2019,11,2019,5107206,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
X = dataset2.iloc[:, 0:553].values
X

array([[5106817, 63140.0, 66300.0, ..., 0, 0, 0],
       [5106817, 62500.0, 66300.0, ..., 0, 0, 0],
       [5106817, 48100.0, 66300.0, ..., 0, 0, 0],
       ...,
       [5189971, 86000.0, 86000.0, ..., 0, 0, 0],
       [5189971, 86000.0, 86000.0, ..., 0, 0, 0],
       [5189971, 86000.0, 86000.0, ..., 0, 0, 0]], dtype=object)

In [14]:
Y = dataset2.iloc[:, 553].values
Y

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [15]:
#split into training and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=11915064)

Linear Regression ML model

In [16]:
#Linear regression ML model
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [17]:
LRmodel = LinearRegression()  
LRmodel.fit(X_train, y_train)

ValueError: could not convert string to float: '618 days 00:00:00.000000000'

In [ ]:
y_pred = LRmodel.predict(X_test)

XGBoost model

In [18]:
#XGBoost model
XGBmodel = XGBClassifier()
XGBmodel.fit(X_train, y_train)

ValueError: could not convert string to float: '618 days 00:00:00.000000000'

In [ ]:
y_pred = XGBmodel.predict(X_test)

In [ ]:
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)